Seojin

# Notice

You can visualize plot only in mac. I tested vedo in Ubuntu, but It did not work.

# Vedo(Visualization of эD Objects)

Vedo is scientific tool for visualizing 3d objects

Official site: https://vedo.embl.es  
Github: https://github.com/marcomusy/vedo/tree/master/examples/notebooks

# Mesh

For visualizing 3d Object, You should make mesh(vertexes) based on nifti image. For making vtk file, I used nii2mesh.

Github: https://github.com/neurolabusc/nii2mesh

ex) nii2mesh lt_hippo_fan.nii lt_hippo_fan_d.vtk

# Library

In [1]:
import nilearn.plotting
import nibabel as nb
import vedo
from matplotlib import cm
import numpy as np
import os

# Custom function

custom function for converting coordinate system.


In [2]:
def LPSp_toRASp(xyz):
    """
    Convert LPS+ coordinate to RAS+ coordinate
    
    :param xyz: LPS+ coord(list)
    
    return xyz(list)
    """
    x = xyz[0]
    y = xyz[1]
    z = xyz[2]
    
    return -x, -y, z

def RASp_toLPSp(xyz):
    """
    Convert RAS+ coordinate to LPS+ coordinate
    
    :param xyz: RAS+ coord(list)
    
    return xyz(list)
    """
    x = xyz[0]
    y = xyz[1]
    z = xyz[2]
    
    return -x, -y, z

def reference2imageCoord(xyz, affine):
    """
    change reference coordinate to image coordinate
    reference coordinate can be scanner coordinate or MNI coordinate...
    
    :param xyz: anatomical coordinate(np.array)
    :param affine: affine matrix(np.array)
    
    return image coordinate(np.array)
    """
    
    result = np.matmul(np.linalg.inv(affine), [xyz[0], xyz[1], xyz[2], 1])[0:3]
    result = np.ceil(result).astype(int) # note: This is ad-hoc process - (np.ceil)
    return result

# Paths

In [3]:
dir_path = "/Users/clmn/Desktop/GP/scripts/GP_vis"

lt_putamen_vtk_path = f"{dir_path}/putamen_caudateNucleus_left_HO.vtk"
rt_putamen_vtk_path = f"{dir_path}/putamen_caudateNucleus_right_HO.vtk"

# os.system(f"nii2mesh {lt_putamen_nii_path} -i b {lt_putamen_vtk_path}")
# os.system(f"nii2mesh {rt_putamen_nii_path} -i b {rt_putamen_vtk_path}")

target_vtk_volume_paths = [
    lt_putamen_vtk_path,
    rt_putamen_vtk_path
]

# Stat map
stat_file_path = f"{dir_path}/Coef.M1_cTBS-GA.nii"
stat = nb.load(stat_file_path)
stat_affine = stat.affine
print("stat-map shape:", stat.shape)

stat-map shape: (77, 92, 77)


# Load Data

In [4]:
# Load Vertex files - (Before skipping next part, you need to identify coordinate about data. 
# In my case, they are RAS+ coordinate
# target_vtk_volumes - volume for visualizing stat
target_vtk_volumes = [vedo.load(vtk_path) for vtk_path in target_vtk_volume_paths]

# Merge target volumes
target_vtk_volume = vedo.mesh.merge(target_vtk_volumes)

t_stat_index = 1
# t_stats = stat.get_fdata()[:,:,:,0, t_stat_index] # Select t-stat
t_stats = stat.get_fdata()

# Visualize configuration


In [5]:
# Color map
color_len = 16

jet = cm.get_cmap('bwr')
jet_colors = jet(np.linspace(0, 1, color_len))[:,:-1]

alphas = np.repeat(1, color_len) # np.arange(1, 1, 0.1) # color Alpha

# Merge target volumes

In [6]:
target_stats = []
for i, vertex in enumerate(target_vtk_volume.points()):
    RASp_coord = vertex
    image_coord = reference2imageCoord(RASp_coord, 
                                       affine = stat_affine).astype(int)

    target_stats.append(t_stats[image_coord[0], image_coord[1], image_coord[2]])
    

In [7]:
# reverse y-axis
# target_vtk_volume = target_vtk_volume.clone(deep=False).mirror("y")

# Show

In [ ]:
# Apply all configuration on volume
target_vtk_volume.cmap(jet_colors, target_stats, alpha=alphas, vmin=-0.5, vmax=0.5).addScalarBar()

# Show
vedo.show(target_vtk_volume, __doc__, viewup="z", axes=0).close()